<a href="https://colab.research.google.com/github/TomasMrkva/project/blob/colab-notebooks/Streamlit_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi -L 

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-393540a8-2e44-0109-479a-02db871a2e50)


In [3]:
#@markdown #**Install libraries**

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

!pip install -q stqdm &> /dev/null
!pip install -q git+https://github.com/TomasMrkva/DifferentiableSketching.git &> /dev/null
!pip install -q torch>=1.7.0 setuptools>=50.3.1 quickdraw>=0.1.0 numpy>=1.19.2 pillow>=8.0.1 tqdm>=4.54.1 pyx>=0.15 torchvision>=0.8.1 matplotlib>=3.3.2 requests>=2.25.0 ipython>=7.19.0 torchbearer>=0.5.3.dev0 lpips>=0.1.2 scikit-learn>=0.24.1 scikit-image~=0.17.2 celluloid &> /dev/null
!pip install -q git+https://github.com/openai/CLIP.git &> /dev/null
!pip install streamlit -q &> /dev/null

<IPython.core.display.Javascript object>

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 53 kB 1.1 MB/s 


In [ ]:
#@markdown #**Streamlit code**
%%writefile ui.py
import streamlit as st
from random import randint
from streamlit import session_state as state
from io import StringIO
from drawing import predict
import os
import torch
import clip
# from inotify_simple import INotify, flags
import json
# from atomicwrites import atomic_write
from time import sleep
from os.path import exists
from utils import atomic_write


hide_menu_style = """
        <style>
        #MainMenu {visibility: hidden; }
        </style> """
st.markdown(hide_menu_style, unsafe_allow_html=True)

st.header("Text-to-paiting/drawing generation")
st.write("OpenAI's CLIP + Differential Drawing")
myform = st.empty()
summarizations_form = st.empty()
chart_checkbox = st.empty()

@st.cache
def load_clip():
    state.model, _ = clip.load('ViT-B/32', torch.device('cuda'), jit=False)

if 'checkbox_val' not in st.session_state:
    state.checkbox_val = False
    state.uploaded_file = str(randint(0,10000))
    state.lines = 425
    state.iters = 400
    state.prompt = ""
    state.main_screen = True
    state.summarizations_screen = False
    state.running_screen = False
    state.radiobox = ""
    load_clip()

def checkbox_click():
    state.checkbox_val = not state.checkbox_val

def main_screen_submit(lines, iters, prompt, summarizations=False):
    chart_checkbox.empty()
    myform.empty()
    state.lines = lines
    state.iters = iters
    state.prompt = prompt
    state.main_screen = False
    if summarizations:
        state.summarizations_screen = True
    else:
        state.running_screen = True
    main()

def summarizations_screen_submit():
    state.main_screen = False
    state.summarizations_screen = False
    state.running_screen = True
    state.prompt = state.radiobox
    main()

def restart():
    state.prompt = ""
    state.main_screen = True
    main()

def main():
    if state.main_screen:
        state.summarizations_screen = False
        state.running_screen = False
    elif state.summarizations_screen:
        state.running_screen = False

if state.main_screen:
    chart_checkbox.checkbox('Insert book paragraph or chapter', value=state.checkbox_val, on_change=checkbox_click)
    if state.checkbox_val:
        with myform.form(key='form-charts'):
            lines = st.slider("Number of lines", value = state.lines, min_value = 100, max_value = 850, step = 10)
            iters = st.slider("Number of iterations", value = state.iters, min_value = 100, max_value = 1000, step = 10)
            prompt = st.text_area("Paste the chapter contents here...", height=200)
            uploaded_file = st.file_uploader("or upload .txt file", type="txt", key=st.session_state.uploaded_file)
            submit = st.form_submit_button("Submit")
        if submit:
            string = ""
            if not prompt and uploaded_file is None:
                st.error('Please insert a chapter content.')
                st.stop()
            elif uploaded_file is not None:
                stringio = StringIO(uploaded_file.getvalue().decode("utf-8"))
                string_data = stringio.read()
                state.uploaded_file = str(randint(0,10000))
                string = string_data
            else:
                string = prompt
            main_screen_submit(lines, iters, string, summarizations=True)
    else:
        with myform.form(key='form-custom'):
            lines = st.slider("Number of lines", value = state.lines, min_value = 100, max_value = 850, step = 10)
            iters = st.slider("Number of iterations", value = state.iters, min_value = 100, max_value = 1000, step = 10)
            prompt = st.text_input("Insert a text prompt for the drawing here", max_chars=100)
            submit = st.form_submit_button("Submit")
        if submit:
            if not prompt:
                st.error('Please insert a prompt.')
                st.stop()
            main_screen_submit(lines, iters, prompt)

if state.summarizations_screen:
    with st.spinner('Generating the prompts from the text...'):
        atomic_write(state.prompt,'drive/MyDrive/3rd_yr_project/text_to_summarise.txt')
        while os.stat("drive/MyDrive/3rd_yr_project/prompts.json").st_size == 0:
          sleep(2)
        sleep(2)
        with open('drive/MyDrive/3rd_yr_project/prompts.json', 'r+') as f:
            data = json.load(f)
            f.truncate(0)
        if (len(data) != 4): print('error')
        headline = data['headline']
        yake_kw = data['yake']
        rake_kw = data['rake']
        bert_kw = data['bert']
    with summarizations_form.form(key='summarizations_form'):
        prompt = st.radio(
            "Choose one of the generated prompts",
            (headline, yake_kw, rake_kw, bert_kw), key='radiobox')
        st.form_submit_button("Submit", on_click=summarizations_screen_submit)
    st.button('Back', on_click=restart)

if state.running_screen:
    summarizations_form.empty()
    clip_sim = predict(state.model, prompt=state.prompt, update={}, lines=state.lines, iters=state.iters)
    st.success('Done, the clip similarity for a best image was: ' + str(clip_sim))
    st.button("Try again", on_click=restart)
    st.stop()

if __name__ == "__main__":
    main()

In [ ]:
!streamlit run ui.py & npx localtunnel --port 8501

2022-04-05 22:44:17.190 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.463s
your url is: https://selfish-wombat-75.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.224.176.36:8501

Generating image for a prompt: older brother called dudley | Parameters updated: {}
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Generating an image for prompt older brother called dudley: 100% 400/400 [02:47<00:00,  2.39it/s]
Generating image for a prompt: The Boy Who Lives | Parameters updated: {}
Generating an image for prompt: The Boy Who Lives:  93% 928/1000 [20:19<01:34,  1.31s/it]

In [ ]:
#TODO: 
#keyword keyphrase extraction, KeyBERT
#PDF make wider lines
#Different transforms
#LR annealing
#videos